In [1]:
import sys  
sys.path.insert(0, './../')
import numpy as np
import bbdc2021 as bbdc
import pandas as pd
from importlib import reload
reload(bbdc)
import tensorflow as tf
from keras import layers
import keras
from keras import backend as K
from itertools import groupby

Using TensorFlow backend.


In [3]:
X_train, Y_train, timepoints, trainDataframe = bbdc.load_data("train.csv", "dataset_fft/")
X_validation, Y_validation, timepoints, validationDataframe = bbdc.load_data("validation.csv", "dataset_fft/")  
X_test, Y_test, timepoints, testDataframe = bbdc.load_data("test.csv", "dataset_fft/")

X_train_cropped = [np.delete(x,1,0) for x in X_train] #TODO besser lösen
Y_train_cropped = [np.delete(x,1,0) for x in Y_train] #TODO besser lösen
X_validation_cropped = [np.delete(x,1,0) for x in X_validation] #TODO besser lösen
Y_validation_cropped = [np.delete(x,1,0) for x in Y_validation] #TODO besser lösen
X_test_cropped = [np.delete(x,1,0) for x in X_test] #TODO besser lösen
Y_test_cropped = [np.delete(x,1,0) for x in Y_test] #TODO besser lösen
inputShape = X_train_cropped[0].shape

In [6]:
channels = [32, 64, 128]

#input_layer = layers.Input(shape=(296,296,1))
input_layer = layers.Input(shape=(inputShape))
##############################
x = input_layer
# Encoder Start
x = layers.Conv1D(channels[0], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[0], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
sh1 = layers.ReLU()(x)

x = layers.MaxPooling1D()(sh1)
#############################
x = layers.Conv1D(channels[1], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[1], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
sh2 = layers.ReLU()(x)
x = layers.MaxPooling1D()(sh2)
#############################
x = layers.Conv1D(channels[2], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[2], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
#Decoder start
x = layers.UpSampling1D(2)(x)

############################
#sh2 = layers.Cropping1D(cropping=10)(sh2)
x = layers.Concatenate()([x,sh2])
x = layers.Conv1D(channels[1], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[1], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.UpSampling1D(2)(x)
#############################
#sh1 = layers.Cropping1D(cropping=16)(sh1)
x = layers.Concatenate()([x,sh1])
x = layers.Conv1D(channels[0], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[0], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(13, kernel_size=1, activation="softmax", padding='same')(x)

model = keras.models.Model(inputs=input_layer, outputs=x)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 208, 17)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 208, 32)      1664        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 208, 32)      128         conv1d_1[0][0]                   
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 208, 32)      0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [32]:
#Mit crossentropy
model = keras.models.Model(inputs=input_layer, outputs=x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['mae'])
history = model.fit(np.array(X_train_cropped),np.array(Y_train_cropped), batch_size=15, epochs=15, validation_data=(np.array(X_validation_cropped), np.array(Y_validation_cropped)))

Train on 8000 samples, validate on 1000 samples
Epoch 1/15
8000/8000 [==============================] - 73s 9ms/step - loss: 0.7556 - mae: 0.0485 - val_loss: 2.0145 - val_mae: 0.1110
Epoch 2/15
8000/8000 [==============================] - 61s 8ms/step - loss: 0.6370 - mae: 0.0419 - val_loss: 0.9144 - val_mae: 0.0594
Epoch 3/15
 795/8000 [=>............................] - ETA: 51s - loss: 0.5913 - mae: 0.0391

KeyboardInterrupt: 

In [29]:
#Mit Dice
model = keras.models.Model(inputs=input_layer, outputs=x)
model.compile(optimizer='adam', loss=bbdc.dice_loss, metrics=['mae'])
history = model.fit(np.array(X_train_cropped),np.array(Y_train_cropped), batch_size=15, epochs=6, validation_data=(np.array(X_validation_cropped), np.array(Y_validation_cropped)))

Train on 8000 samples, validate on 1000 samples
Epoch 1/6
8000/8000 [==============================] - 67s 8ms/step - loss: 0.3347 - mae: 0.1535 - val_loss: 0.3354 - val_mae: 0.1534
Epoch 2/6
6480/8000 [=======================>......] - ETA: 10s - loss: 0.3344 - mae: 0.1536

KeyboardInterrupt: 

In [ ]:
#Mit binary_crossentropy
model = keras.models.Model(inputs=input_layer, outputs=x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mae'])
history = model.fit(np.array(X_train),np.array(Y_train), batch_size=15, epochs=15, validation_data=(np.array(X_validation), np.array(Y_validation)))

In [33]:
prediction = model.predict(np.array(X_test_cropped))

In [37]:
from operator import itemgetter
y_predicted = np.argmax(prediction, axis=2)
y_predictedNew = np.zeros((y_predicted.shape[0]+1, y_predicted.shape[1]))
y_predictedNew[0] = timepoints[1:]
y_predictedNew[1:] = y_predicted


result = np.transpose(y_predictedNew[0:2])
#print(result)
#result 
a = [list(group) for key, group in groupby(result, itemgetter(1))]
a

[[array([0.048, 0.   ]),
  array([0.096, 0.   ]),
  array([0.144, 0.   ]),
  array([0.192, 0.   ]),
  array([0.24, 0.  ]),
  array([0.288, 0.   ]),
  array([0.336, 0.   ]),
  array([0.384, 0.   ]),
  array([0.432, 0.   ]),
  array([0.48, 0.  ]),
  array([0.528, 0.   ]),
  array([0.576, 0.   ]),
  array([0.624, 0.   ]),
  array([0.672, 0.   ]),
  array([0.72, 0.  ]),
  array([0.768, 0.   ]),
  array([0.816, 0.   ]),
  array([0.864, 0.   ]),
  array([0.912, 0.   ]),
  array([0.96, 0.  ]),
  array([1.008, 0.   ]),
  array([1.056, 0.   ]),
  array([1.104, 0.   ]),
  array([1.152, 0.   ]),
  array([1.2, 0. ])],
 [array([1.248, 5.   ])],
 [array([1.296, 1.   ])],
 [array([1.344, 5.   ]),
  array([1.392, 5.   ]),
  array([1.44, 5.  ]),
  array([1.488, 5.   ]),
  array([1.536, 5.   ]),
  array([1.584, 5.   ]),
  array([1.632, 5.   ]),
  array([1.68, 5.  ]),
  array([1.728, 5.   ]),
  array([1.776, 5.   ]),
  array([1.824, 5.   ]),
  array([1.872, 5.   ]),
  array([1.92, 5.  ]),
  array([1.968,

In [38]:
inv_map = {v: k for k, v in bbdc.LABEL_DICT.items()}
print(y_predicted[0])
[inv_map[key] for key in y_predicted[0]]

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  5  1  5  5  5  5  5  5  5  5  5  5  5  5  5  5  5  7  7  7  7  7  7
  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  7  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4
  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4  4 12 12 12 12  9  9
  9  9  9  9  9  9  9  9  0  9  0  0  0  0  0  0  0  0  0  0  0  7  7  0
  0  0  7  7  7  7  7  7  7  7  0  0  7  0  0  0  0  0  0  0  0  0  7  7
  7  9  9  9  9  9  9  9  9  9  9  9  7  7  7  7  9  9  9  9  9  9  9  9
  9  9  9  9  9  9  9  9  9  0  0  0  0  0  0  0]


['Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Church_bell',
 'Bark',
 'Church_bell',
 'Church_bell',
 'Church_bell',
 'Church_bell',
 'Church_bell',
 'Church_bell',
 'Church_bell',
 'Church_bell',
 'Church_bell',
 'Church_bell',
 'Church_bell',
 'Church_bell',
 'Church_bell',
 'Church_bell',
 'Church_bell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Doorbell',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Noise',
 'Cheering',
 'Cheering',
 'Cheering